In [4]:
import requests
import json

# Some utilities for flattening the explain into something a bit more
# readable. Pass Explain JSON, get something readable (ironically this is what Solr's default output is :-p)
def flatten(l):
    [item for sublist in l for item in sublist]

def simplerExplain(explainJson, depth=0):
    result = " " * (depth * 2) + "%s, %s\n" % (explainJson['value'], explainJson['description'])
    #print json.dumps(explainJson, indent=True)
    if 'details' in explainJson:
        for detail in explainJson['details']:
            result += simplerExplain(detail, depth=depth+1)
    return result

# Create a stub keywords.txt at /tmp
keywordsPath = "/tmp/keywords.txt";
import os
if os.name == "nt":
    keywordsPath = "C:\\temp\keywords.txt"

f = open("/tmp/keywords.txt", "w")
f.write("maine\n")
f.close()


# 4.2.2

In [82]:
settings = {
  "settings": {
    "analysis": {
      "analyzer": {
        "standard_clone": {
          "tokenizer": "standard",
          "filter": [
            "standard",
            "lowercase",
            "stop "]}}}}}

requests.delete("http://localhost:9200/my_library")
requests.put("http://localhost:9200/my_library", data=json.dumps(settings))

<Response [200]>

In [97]:
resp = requests.get("http://localhost:9200/my_library/_analyze?analyzer=standard_clone", 
                    data="Dr. Strangelove: Or How I Learned to Stop Worrying and Love the Bomb")
resp = json.loads(resp.text)
",".join(['[' + token['token'] + ']'  for token in resp['tokens']])

u'[dr],[strangelove],[how],[i],[learned],[stop],[worrying],[love],[bomb]'

In [98]:
resp = requests.get("http://localhost:9200/my_library/_analyze?analyzer=standard_clone", 
                    data="mr. weirdlove: don't worry, I'm learning to start loving bombs")
resp = json.loads(resp.text)
",".join(['[' + token['token'] + ']'  for token in resp['tokens']])

u"[mr],[weirdlove],[don't],[worry],[i'm],[learning],[start],[loving],[bombs]"

In [113]:
settings = {
  "settings": {
    "analysis": {
      "filter": {
        "english_stop": {
          "type":       "stop",
          "stopwords":  "_english_"},
        "english_keywords": {
          "type":       "keyword_marker",
          "keywords_path":   keywordsPath},
        "english_stemmer": {
          "type":       "stemmer",
          "language":   "english"},
        "english_possessive_stemmer": {
          "type":       "stemmer",
          "language":   "possessive_english"}},
      "analyzer": {
        "english_clone": {
          "tokenizer":  "standard",
          "filter": [
            "standard",  
            "english_possessive_stemmer",
            "lowercase",
            "english_stop",
            "english_keywords",
            "english_stemmer"]}}}}}

In [114]:
requests.delete("http://localhost:9200/my_library")
requests.put("http://localhost:9200/my_library", data=json.dumps(settings)).text

u'{"acknowledged":true}'

In [115]:
resp = requests.get("http://localhost:9200/my_library/_analyze?analyzer=english_clone", 
                    data="flowers flower flowered flower")
resp = json.loads(resp.text)
",".join(['[' + token['token'] + ']'  for token in resp['tokens']])

u'[flower],[flower],[flower],[flower]'

In [116]:
resp = requests.get("http://localhost:9200/my_library/_analyze?analyzer=english_clone", 
                    data="silly silliness sillied sillying")
resp = json.loads(resp.text)
",".join(['[' + token['token'] + ']'  for token in resp['tokens']])

u'[silli],[silli],[silli],[silli]'

In [117]:
resp = requests.get("http://localhost:9200/my_library/_analyze?analyzer=english_clone", 
                    data="maine main")
resp = json.loads(resp.text)
",".join(['[' + token['token'] + ']'  for token in resp['tokens']])

u'[maine],[main]'

In [118]:
resp = requests.get("http://localhost:9200/my_library/_analyze?analyzer=english_clone", 
                    data="Dr. Strangelove: Or How I Learned to Stop Worrying and Love the Bomb")
resp = json.loads(resp.text)
",".join(['[' + token['token'] + ']'  for token in resp['tokens']])

u'[dr],[strangelov],[how],[i],[learn],[stop],[worri],[love],[bomb]'

In [119]:
resp = requests.get("http://localhost:9200/my_library/_analyze?analyzer=english_clone", 
                    data="mr. weirdlove: don't worry, I'm learning to start loving bombs")
resp = json.loads(resp.text)
",".join(['[' + token['token'] + ']'  for token in resp['tokens']])

u"[mr],[weirdlov],[don't],[worri],[i'm],[learn],[start],[love],[bomb]"

# 4.2.3

In [128]:
# Requires phonetic analysis plugin 
# https://www.elastic.co/guide/en/elasticsearch/plugins/2.1/analysis-phonetic.html
#
# To install on *nix systems
# 1. Navigate to Elasticsearc directory
# cd /usr/share/elasticsearch
# 2. sudo bin/plugin install analysis-phonetic

In [130]:
settings = { "settings": {
    "analysis": {
      "analyzer": {
        "phonetic": {
          "tokenizer": "standard",
          "filter": [
            "standard",
            "lowercase",
            "my_doublemetaphone"]}},
      "filter": {
        "my_doublemetaphone": {
          "type": "phonetic",
          "encoder": "doublemetaphone", 
          "replace": True}}}}}


In [131]:
requests.delete("http://localhost:9200/my_library")
requests.put("http://localhost:9200/my_library", data=json.dumps(settings)).text

u'{"acknowledged":true}'

In [133]:
resp = requests.get("http://localhost:9200/my_library/_analyze?analyzer=phonetic", 
                    data="message from Dalai Lama")
resp = json.loads(resp.text)
",".join(['[' + token['token'] + ']'  for token in resp['tokens']])

u'[MSJ],[MSK],[FRM],[TL],[LM]'

In [134]:
resp = requests.get("http://localhost:9200/my_library/_analyze?analyzer=phonetic", 
                    data="massage from tall llama")
resp = json.loads(resp.text)
",".join(['[' + token['token'] + ']'  for token in resp['tokens']])

u'[MSJ],[MSK],[FRM],[TL],[LM]'

# 4.3.1

In [4]:
import json
requests.delete("http://localhost:9200/my_library")

settings = {"settings": 
    {"number_of_shards": 1}}

requests.put("http://localhost:9200/my_library", data=json.dumps(settings))

<Response [200]>

In [5]:
import json

documents = [
{ "title":"apple apple apple apple apple"},
{ "title":"apple apple apple banana banana"},
{ "title":"apple banana blueberry coconut"},
]

for idx, document in enumerate(documents):
    URL = "http://localhost:9200/my_library/example/%s" % (idx + 1)
    print URL
    print document
    resp = requests.put(URL, data=json.dumps(document))

http://localhost:9200/my_library/example/1
{'title': 'apple apple apple apple apple'}
http://localhost:9200/my_library/example/2
{'title': 'apple apple apple banana banana'}
http://localhost:9200/my_library/example/3
{'title': 'apple banana blueberry coconut'}


In [6]:
search = { "explain": "true",
  "query": {
    "match": {
      "title": "apple"}}}

resp = requests.get("http://localhost:9200/my_library/example/_search?pretty=true", data=json.dumps(search))
respJson = json.loads(resp.text)
for idx, hit in enumerate(respJson['hits']['hits']):
    print ("%s.  " % (idx + 1)) + repr(hit['_source'])
    print simplerExplain(hit['_explanation'])

1.  {u'title': u'apple apple apple apple apple'}
0.6968462, sum of:
  0.6968462, weight(title:apple in 0) [PerFieldSimilarity], result of:
    0.6968462, fieldWeight in 0, product of:
      2.236068, tf(freq=5.0), with freq of:
        5.0, termFreq=5.0
      0.71231794, idf(docFreq=3, maxDocs=3)
      0.4375, fieldNorm(doc=0)
  0.0, match on required clause, product of:
    0.0, # clause
    1.4038675, _type:example, product of:
      1.0, boost
      1.4038675, queryNorm

2.  {u'title': u'apple apple apple banana banana'}
0.5397748, sum of:
  0.5397748, weight(title:apple in 1) [PerFieldSimilarity], result of:
    0.5397748, fieldWeight in 1, product of:
      1.7320508, tf(freq=3.0), with freq of:
        3.0, termFreq=3.0
      0.71231794, idf(docFreq=3, maxDocs=3)
      0.4375, fieldNorm(doc=1)
  0.0, match on required clause, product of:
    0.0, # clause
    1.4038675, _type:example, product of:
      1.0, boost
      1.4038675, queryNorm

3.  {u'title': u'apple banana blueberry

In [7]:
extraDoc = { "title":"apples apple"}
resp = requests.put("http://localhost:9200/my_library/example/%s" % (5), data=json.dumps(extraDoc))

In [8]:
resp = requests.get("http://localhost:9200/my_library/example/_search?pretty=true", data=json.dumps(search))
respJson = json.loads(resp.text)
for idx, hit in enumerate(respJson['hits']['hits']):
    print ("%s.  " % (idx + 1)) + repr(hit['_source'])
    print simplerExplain(hit['_explanation'])

1.  {u'title': u'apple apple apple apple apple'}
0.6968462, sum of:
  0.6968462, weight(title:apple in 0) [PerFieldSimilarity], result of:
    0.6968462, fieldWeight in 0, product of:
      2.236068, tf(freq=5.0), with freq of:
        5.0, termFreq=5.0
      0.71231794, idf(docFreq=3, maxDocs=3)
      0.4375, fieldNorm(doc=0)
  0.0, match on required clause, product of:
    0.0, # clause
    1.4038675, _type:example, product of:
      1.0, boost
      1.4038675, queryNorm

2.  {u'title': u'apple apple apple banana banana'}
0.5397748, sum of:
  0.5397748, weight(title:apple in 1) [PerFieldSimilarity], result of:
    0.5397748, fieldWeight in 1, product of:
      1.7320508, tf(freq=3.0), with freq of:
        3.0, termFreq=3.0
      0.71231794, idf(docFreq=3, maxDocs=3)
      0.4375, fieldNorm(doc=1)
  0.0, match on required clause, product of:
    0.0, # clause
    1.4038675, _type:example, product of:
      1.0, boost
      1.4038675, queryNorm

3.  {u'title': u'apple banana blueberry

In [9]:
requests.delete("http://localhost:9200/my_library")

settings = {"settings": 
    {"number_of_shards": 1, 
     "index": 
         {"analysis": {
            "analyzer": {
              "default": {
                "type": "english"
             
    }}}}}
}

resp = requests.put("http://localhost:9200/my_library", data=json.dumps(settings))
print resp.text

{"acknowledged":true}


In [10]:
documents.append(extraDoc)

for idx, document in enumerate(documents):
    URL = "http://localhost:9200/my_library/example/%s" % (idx + 1)
    print URL
    print document
    resp = requests.put(URL, data=json.dumps(document))

http://localhost:9200/my_library/example/1
{'title': 'apple apple apple apple apple'}
http://localhost:9200/my_library/example/2
{'title': 'apple apple apple banana banana'}
http://localhost:9200/my_library/example/3
{'title': 'apple banana blueberry coconut'}
http://localhost:9200/my_library/example/4
{'title': 'apples apple'}


In [11]:
search = { "explain": "true",
  "query": {
    "match": {
      "title": "apple"}}}

resp = requests.get("http://localhost:9200/my_library/example/_search?pretty=true", data=json.dumps(search))
respJson = json.loads(resp.text)
for hit in respJson['hits']['hits']:
    print hit['_source']
    print simplerExplain(hit['_explanation'])

{u'title': u'apple apple apple apple apple'}
0.75998294, sum of:
  0.75998294, weight(title:appl in 0) [PerFieldSimilarity], result of:
    0.75998294, fieldWeight in 0, product of:
      2.236068, tf(freq=5.0), with freq of:
        5.0, termFreq=5.0
      0.7768564, idf(docFreq=4, maxDocs=4)
      0.4375, fieldNorm(doc=0)
  0.0, match on required clause, product of:
    0.0, # clause
    1.2872392, _type:example, product of:
      1.0, boost
      1.2872392, queryNorm

{u'title': u'apples apple'}
0.6866506, sum of:
  0.6866506, weight(title:appl in 3) [PerFieldSimilarity], result of:
    0.6866506, fieldWeight in 3, product of:
      1.4142135, tf(freq=2.0), with freq of:
        2.0, termFreq=2.0
      0.7768564, idf(docFreq=4, maxDocs=4)
      0.625, fieldNorm(doc=3)
  0.0, match on required clause, product of:
    0.0, # clause
    1.2872392, _type:example, product of:
      1.0, boost
      1.2872392, queryNorm

{u'title': u'apple apple apple banana banana'}
0.58868027, sum of:
 

In [12]:
search = { "explain": "true",
  "query": {
    "match": {
      "title": "apple banana"}}}

resp = requests.get("http://localhost:9200/my_library/example/_search?pretty=true", data=json.dumps(search))
respJson = json.loads(resp.text)
for idx, hit in enumerate(respJson['hits']['hits']):
    print ("%s.  " % (idx + 1)) + repr(hit['_source'])
    print simplerExplain(hit['_explanation'])

1.  {u'title': u'apple apple apple banana banana'}
0.9862758, sum of:
  0.9862758, sum of:
    0.30409503, weight(title:appl in 1) [PerFieldSimilarity], result of:
      0.30409503, score(doc=1,freq=3.0), product of:
        0.5165708, queryWeight, product of:
          0.7768564, idf(docFreq=4, maxDocs=4)
          0.6649502, queryNorm
        0.58868027, fieldWeight in 1, product of:
          1.7320508, tf(freq=3.0), with freq of:
            3.0, termFreq=3.0
          0.7768564, idf(docFreq=4, maxDocs=4)
          0.4375, fieldNorm(doc=1)
    0.68218076, weight(title:banana in 1) [PerFieldSimilarity], result of:
      0.68218076, score(doc=1,freq=2.0), product of:
        0.85624444, queryWeight, product of:
          1.287682, idf(docFreq=2, maxDocs=4)
          0.6649502, queryNorm
        0.79671264, fieldWeight in 1, product of:
          1.4142135, tf(freq=2.0), with freq of:
            2.0, termFreq=2.0
          1.287682, idf(docFreq=2, maxDocs=4)
          0.4375, fieldNo

## 4.4.1 -- Acronyms

In [98]:
data = {
  "settings": {
    "analysis": {
      "filter": {
        "acronyms": {
          "type": "word_delimiter",
          "catenate_all": True,
          "generate_word_parts": False,
          "generateNumberParts": False}},
      "analyzer": {
        "standard_with_acronyms": {
          "tokenizer": "whitespace",
          "filter": ["standard","lowercase","acronyms"]}}}}}

requests.delete("http://localhost:9200/example")
resp = requests.post("http://localhost:9200/example", data=json.dumps(data))
print resp

<Response [200]>


In [99]:
resp = requests.get("http://localhost:9200/example/_analyze?analyzer=standard_with_acronyms&format=yaml", 
                     data="I.B.M. versus IBM versus ibm")
print resp.text

---
tokens:
- token: "ibm"
  start_offset: 0
  end_offset: 5
  type: "word"
  position: 0
- token: "versus"
  start_offset: 7
  end_offset: 13
  type: "word"
  position: 1
- token: "ibm"
  start_offset: 14
  end_offset: 17
  type: "word"
  position: 2
- token: "versus"
  start_offset: 18
  end_offset: 24
  type: "word"
  position: 3
- token: "ibm"
  start_offset: 25
  end_offset: 28
  type: "word"
  position: 4



In [100]:
resp = requests.get("http://localhost:9200/example/_analyze?analyzer=standard_with_acronyms&format=yaml", 
                     data="1.800.867.5309")
print resp.text
resp = requests.get("http://localhost:9200/example/_analyze?analyzer=standard_with_acronyms&format=yaml", 
                     data="1-800-867-5309")
print resp.text

---
tokens:
- token: "18008675309"
  start_offset: 0
  end_offset: 14
  type: "word"
  position: 0

---
tokens:
- token: "18008675309"
  start_offset: 0
  end_offset: 14
  type: "word"
  position: 0



In [25]:
data = {
  "settings": {
    "analysis": {
      "filter": {
        "phone_num_filter": {
          "type": "word_delimiter",
          "catenate_all": True,
          "generate_number_parts": False},
        "phone_num_parts": {
          "type": "pattern_capture",
          "patterns": ["(\\d{7}$)","(\\d{10}$)"],
          "preserve_original": True}},
      "analyzer": {
        "phone_num": {
          "tokenizer": "keyword",
          "filter": ["phone_num_filter","phone_num_parts"]}}}}}
requests.delete("http://localhost:9200/example")
resp = requests.post("http://localhost:9200/example", data=json.dumps(data))
print resp

<Response [200]>


In [26]:
resp = requests.get("http://localhost:9200/example/_analyze?analyzer=phone_num&format=yaml", 
                     data="1(800)867-5309")
print resp.text

---
tokens:
- token: "18008675309"
  start_offset: 0
  end_offset: 14
  type: "word"
  position: 0
- token: "8008675309"
  start_offset: 0
  end_offset: 14
  type: "word"
  position: 0
- token: "8675309"
  start_offset: 0
  end_offset: 14
  type: "word"
  position: 0



# 4.4.2 Capturing Meaning with Synonyms

In [62]:
data = {
  "settings": {
    "analysis": {
      "filter": {
        "english_stop": {
          "type":       "stop",
          "stopwords":  "_english_"},
        "english_keywords": {
          "type":       "keyword_marker",
          "keywords_path":   "/tmp/keywords.txt"},
        "english_stemmer": {
          "type":       "stemmer",
          "language":   "english"},
        "english_possessive_stemmer": {
          "type":       "stemmer",
          "language":   "possessive_english"},
        "retail_syn_filter": {  # 1. our new filter
          "type": "synonym",
          "synonyms": [
            "dress shoe,dress shoes => dress_shoe, shoe"]
         }
      },
      "analyzer": {
        "retail_analyzer": { # 2. our new analyzer
          "tokenizer": "standard",
          "filter":[
            "english_possessive_stemmer",
            "lowercase",
            "retail_syn_filter",  # 2. here’s where the filter is used 
            "english_stop",
            "english_keywords",
            "english_stemmer"]}}}},
  "mappings": {
    "items": {
      "properties": {
        "desc": {  # 3. our desc field includes synonyms
          "type": "string",
          "analyzer": "retail_analyzer"}}}}}
        
requests.delete("http://localhost:9200/retail")
print resp.status_code
print resp.text
resp = requests.put("http://localhost:9200/retail", data=json.dumps(data))
print resp.status_code
print resp.text

200
{"acknowledged":true}
200
{"acknowledged":true}


In [63]:
data = { "desc": "bob's brand dress shoes are the bomb diggity"} # “dress shoe” document
requests.post("http://localhost:9200/retail/items/1",  
             data=json.dumps(data))

data = { "desc": "this little black dress is sure to impress"}
requests.post("http://localhost:9200/retail/items/2",  # “dress” document
             data=json.dumps(data))

data = { "desc": "tennis shoes... you know, for tennis"}
requests.post("http://localhost:9200/retail/items/3",  # “shoe” document
             data=json.dumps(data))

<Response [201]>

In [64]:
search = {
    "query": {
        "match": {
          "desc": "dress"
        }
    }
}
resp= requests.get("http://localhost:9200/retail/_search?format=yaml", data=json.dumps(search))
print resp.text                   
resp= requests.get("http://localhost:9200/retail/_validate/query?explain&format=yaml", data=json.dumps(search))
print resp.text

---
took: 2
timed_out: false
_shards:
  total: 5
  successful: 5
  failed: 0
hits:
  total: 1
  max_score: 0.13424811
  hits:
  - _index: "retail"
    _type: "items"
    _id: "2"
    _score: 0.13424811
    _source:
      desc: "this little black dress is sure to impress"

---
valid: true
_shards:
  total: 1
  successful: 1
  failed: 0
explanations:
- index: "retail"
  valid: true
  explanation: "desc:dress"



In [65]:
search = {
    "query": {
        "match": {
          "desc": "shoe"
        }
    }
}
resp= requests.get("http://localhost:9200/retail/_search?format=yaml", data=json.dumps(search))
print resp.text

---
took: 2
timed_out: false
_shards:
  total: 5
  successful: 5
  failed: 0
hits:
  total: 2
  max_score: 0.13424811
  hits:
  - _index: "retail"
    _type: "items"
    _id: "1"
    _score: 0.13424811
    _source:
      desc: "bob's brand dress shoes are the bomb diggity"
  - _index: "retail"
    _type: "items"
    _id: "3"
    _score: 0.13424811
    _source:
      desc: "tennis shoes... you know, for tennis"



In [45]:
resp= requests.get("http://localhost:9200/retail/_analyze?analyzer=retail_analyzer&format=yaml", 
                   data="bob's brand dress shoes are the bomb diggity")
print resp.text

---
tokens:
- token: "bob"
  start_offset: 0
  end_offset: 5
  type: "<ALPHANUM>"
  position: 0
- token: "brand"
  start_offset: 6
  end_offset: 11
  type: "<ALPHANUM>"
  position: 1
- token: "dress_sho"
  start_offset: 12
  end_offset: 23
  type: "SYNONYM"
  position: 2
- token: "shoe"
  start_offset: 12
  end_offset: 23
  type: "SYNONYM"
  position: 2
- token: "bomb"
  start_offset: 32
  end_offset: 36
  type: "<ALPHANUM>"
  position: 5
- token: "diggiti"
  start_offset: 37
  end_offset: 44
  type: "<ALPHANUM>"
  position: 6



In [46]:
resp= requests.get("http://localhost:9200/retail/_analyze?analyzer=retail_analyzer&format=yaml", 
                   data="tennis shoes... you know, for tennis")
print resp.text

---
tokens:
- token: "tenni"
  start_offset: 0
  end_offset: 6
  type: "<ALPHANUM>"
  position: 0
- token: "shoe"
  start_offset: 7
  end_offset: 12
  type: "<ALPHANUM>"
  position: 1
- token: "you"
  start_offset: 16
  end_offset: 19
  type: "<ALPHANUM>"
  position: 2
- token: "know"
  start_offset: 20
  end_offset: 24
  type: "<ALPHANUM>"
  position: 3
- token: "tenni"
  start_offset: 30
  end_offset: 36
  type: "<ALPHANUM>"
  position: 5



In [73]:
data = {
  "settings": {
    "analysis": {
      "filter": {
        "english_stop": {
          "type":       "stop",
          "stopwords":  "_english_"},
        "english_keywords": {
          "type":       "keyword_marker",
          "keywords_path":   "/tmp/keywords.txt"},
        "english_stemmer": {
          "type":       "stemmer",
          "language":   "english"},
        "english_possessive_stemmer": {
          "type":       "stemmer",
          "language":   "possessive_english"},
         "retail_syn_filter_index": {
          "type": "synonym",
          "synonyms": ["dress shoe,dress shoes => dress_shoe, shoe"]
         },
         "retail_syn_filter_search": {
           "type": "synonym",
           "synonyms": ["dress shoe,dress shoes => dress_shoe"]
          }                 
      },
      "analyzer": {
         "retail_analyzer_index": {
            "tokenizer": "standard",
            "filter": [
              "english_possessive_stemmer",
              "lowercase",
              "retail_syn_filter_index",
              "english_stop",
              "english_keywords",
              "english_stemmer"
            ]},
          "retail_analyzer_search": {
             "tokenizer": "standard",
             "filter": [
               "english_possessive_stemmer",
               "lowercase",
               "retail_syn_filter_search",
                "english_stop",
                "english_keywords",
                "english_stemmer"
            ]}            
            }}},
  "mappings": {
    "items": {
      "properties": {
        "desc": {  # 3. our desc field includes synonyms
          "type": "string",
          "analyzer": "retail_analyzer_index",
          "search_analyzer": "retail_analyzer_search"}}}}}
        
requests.delete("http://localhost:9200/retail")
print resp.status_code
print resp.text
resp = requests.put("http://localhost:9200/retail", data=json.dumps(data))
print resp.status_code
print resp.text

200
{"acknowledged":true}
200
{"acknowledged":true}


## 4.4.3.1 

In [14]:
settings = { "settings": {
    "analysis": {
      "analyzer": {
        "path_hierarchy": {
          "tokenizer": "path_hierarchy"}}}},
  "mappings": {
      "item": {
        "properties": {
          "inventory_dir": {
            "type": "string",
            "analyzer": "path_hierarchy"}}}}}

requests.delete("http://localhost:9200/catalog")
resp = requests.put("http://localhost:9200/catalog", data=json.dumps(settings))
print resp.status_code
print resp.text

data = { "inventory_dir":"/fruit/apples/fuji",
          "description":"crisp, sweet-flavoured, long shelf-life"}
requests.put("http://localhost:9200/catalog/item/1", data=json.dumps(data))


200
{"acknowledged":true}


<Response [201]>

In [19]:
query = \
{ "query": {
    "bool": {
      "should":
        [{"match": {"description": "shelf"}}],
      "filter": [
         {"term": {"inventory_dir": "/fruit/apples/fuji"}}]}}}


resp = requests.get("http://localhost:9200/catalog/_search?format=yaml", data=json.dumps(query))
print resp.text

---
took: 1
timed_out: false
_shards:
  total: 5
  successful: 5
  failed: 0
hits:
  total: 1
  max_score: 0.11506981
  hits:
  - _index: "catalog"
    _type: "item"
    _id: "1"
    _score: 0.11506981
    _source:
      inventory_dir: "/fruit/apples/fuji"
      description: "crisp, sweet-flavoured, long shelf-life"

